In [ ]:
import os
import sys
import torch

import pandas as pd
import numpy as np
import torch.nn as nn
from torch.optim import Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from PIL import Image
from tqdm import tqdm

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from dotenv import load_dotenv, dotenv_values

# 하이드라와 주피터 노트북은 아규먼트 관련 충돌이 발생하므로 초기화 해줌
sys.argv = ['']
# 환경변수 읽기
if (python_path := dotenv_values().get('PYTHONPATH')) and python_path not in sys.path: sys.path.append(python_path)

from src.data.CvImageDataset import get_datasets
from src.model.CustomModel import CustomModel
from src.util import config
from src.util.utils import send_kakao_message

# 시드 고정
def random_seed(seed_num=42):

    """ SEED = seed_num
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True """
    
    # seed_everything 은 위의 내용 제어 + 밑에내용
    pl.seed_everything(seed_num)
    #torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False

# 데이터 준비 함수
def prepare_data(batch_size=32, num_workers=4):
    
   # 데이터셋 생성
    train_dataset, val_dataset, test_dataset = get_datasets()

    # DataLoader 정의
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    val_loader = DataLoader(
        val_dataset,  # 별도의 검증 데이터셋
        batch_size=batch_size,
        shuffle=False,  # 검증 시에는 셔플하지 않음
        num_workers=num_workers,
        pin_memory=True,
        drop_last=False
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    
    return train_loader, val_loader, test_loader


def main():

    # model config
    model_name = 'efficientnet_b4' # 'resnet50' 'efficientnet_b4', ...

    # training config
    EPOCHS = 1
    BATCH_SIZE = 32
    num_workers = 0
    num_classes = 17
    learning_rate = 1e-3
    
    # 모델 초기화 전에 설정
    torch.set_float32_matmul_precision('medium')
    
    random_seed(42)

    # 데이터 로더 준비
    train_loader, val_loader, test_loader = prepare_data(batch_size=BATCH_SIZE, num_workers=num_workers)
    
    model = CustomModel(
        model_name= model_name,
        num_classes=num_classes,
        learning_rate=learning_rate
    )

     # 콜백을 직접 생성
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=5,
            mode='min',
            min_delta=0.001,
            verbose=True
        ),
        LearningRateMonitor(
            logging_interval='epoch',
            log_momentum=False
        )
    ]

    trainer = Trainer(default_root_dir=config.OUTPUTS_DIR, max_epochs=EPOCHS, accelerator='auto', callbacks=callbacks)
    
    # 훈련
    trainer.fit(model, train_loader, val_loader)
    #trainer.fit(model, train_loader, val_loader, ckpt_path=config.OUTPUTS_DIR + "/lightning_logs/version_0/checkpoints/epoch=9-step=790.ckpt")
    #trainer.fit(model, train_loader, val_loader, ckpt_path=config.OUTPUTS_DIR + "/lightning_logs/version_1/checkpoints/epoch=11-step=948.ckpt")
    # 테스트
    trainer.test(model, test_loader)

    print("테스트 갯수=",len(model.test_predictions))
    
    if len(model.test_predictions) > 0:
        # 모든 예측값과 실제값 합치기
        all_preds = model.test_predictions
        
        pred_df = pd.DataFrame(test_loader.dataset.df, columns=['ID', 'target'])
        pred_df['target'] = all_preds

        sample_submission_df = pd.read_csv(config.CV_CLS_TEST_CSV)
        assert (sample_submission_df['ID'] == pred_df['ID']).all()
        pred_df.to_csv(config.OUTPUTS_DIR + "/pred.csv", index=False)

    else:
        print("테스트 결과를 가져올 수 없습니다.")

    send_kakao_message("작업종료!")
    
if __name__ == "__main__":
    main()

Seed set to 42
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | model          | EfficientNet       | 17.6 M | train
1 | train_accuracy | MulticlassAccuracy | 0      | train
2 | train_f1       | MulticlassF1Score  | 0      | train
3 | val_accuracy   | MulticlassAccuracy | 0      | train
4 | val_f1         | MulticlassF1Score  | 0      | train
5 | criterion      | CrossEntropyLoss   | 0      | train
--------------------------------------------------------------
17.6 M    Trainable params
0         Non-trainable params
17.6 M    Total params
70.316    Total

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Metric val_loss improved. New best score: 0.692
`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

테스트 갯수= 3140
명령어: ['python', '/data/ephemeral/home/python_work/module/send_kakao_message.py', '작업종료!\n\n']
반환 코드: 0
표준 출력: /data/ephemeral/home/python_work/module/kakao_token.json에서 토큰 정보를 성공적으로 불러왔습니다.
{"result_code":0}
200

표준 에러: 


In [ ]:
import pandas as pd

data = pd.read_csv("../data/row/train.csv")
value_counts_df = data['target'].value_counts().reset_index(name='count')
value_counts_df

In [ ]:
import pandas as pd

data2 = pd.read_csv("../data/row/meta2.csv")

result = pd.merge(value_counts_df, data2, on='target', how='inner')
result.sort_values(by='target')

In [ ]:
import psutil
import gc
import torch

def monitor_memory():
    process = psutil.Process()
    print(f"메모리 사용량: {process.memory_info().rss / 1024 / 1024:.2f} MB")

monitor_memory()
# 배치 처리 후 메모리 정리
gc.collect()
torch.cuda.empty_cache()  # GPU 메모리 정리